In [1]:
#Import necessary packages
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

#ConcatDataset and Subset are possibly useful when doing semi-supervised
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

#This is for the progress bar
from tqdm import tqdm

In [2]:
#It is important to do data augmentation in training.
#However, not every augmentation is useful
#Please think about what kind of augmentation is helpful for food recognition
train_tfm = transforms.Compose([
    transforms.Resize((256, 256)),
    #You may add some transforms here
    transforms.ColorJitter(0.1, 0.1, 0.1),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.ToTensor(),
    #transforms.Normalize([0.554, 0.450, 0.343], [0.231, 0.241, 0.241]),
    #transforms.RandomErasing()
    
])
#train_tfm = transforms.Compose([
#    transforms.RandomResizedCrop((128, 128)),
#    transforms.RandomChoice(
#        [transforms.AutoAugment(),
#        transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
#        transforms.AutoAugment(transforms.AutoAugmentPolicy.SVHN)]
#    ),
#    transforms.RandomHorizontalFlip(p=0.5),
#    transforms.ColorJitter(brightness=0.5),
#    transforms.RandomRotation(5),
#    transforms.ToTensor(),
#])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    #transforms.Normalize([0.554, 0.450, 0.343], [0.231, 0.241, 0.241])
])

In [3]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

#Construct datasets
#The augment "loader" tells how torchvision reads the data
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)

valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
#train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = 8)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, drop_last = True, num_workers = 8)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

Debugger :  class -> idx 
{'00': 0, '01': 1, '02': 2, '03': 3, '04': 4, '05': 5, '06': 6, '07': 7, '08': 8, '09': 9, '10': 10}
Debugger :  class -> idx 
{'00': 0, '01': 1, '02': 2, '03': 3, '04': 4, '05': 5, '06': 6, '07': 7, '08': 8, '09': 9, '10': 10}
Debugger :  class -> idx 
{'00': 0}
Debugger :  class -> idx 
{'00': 0}


In [4]:
class pDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.data = X
        #y = y.astype(np.int)
        self.label = y
    def __getitem__(self,idx): # if the index is idx, what will be the data?
        #print(self.label)
        return self.data, self.label
    def __len__(self): # What is the length of the dataset
        return 1

In [5]:
def get_pseudo_labels(dataset, model, threshold = 0.65):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #print(device)
    
    #Construct a data loader
    data_loader = DataLoader(dataset, batch_size = batch_size, shuffle = True, drop_last = True, num_workers = 8)
    
    #make sure the model is in eval mode
    model.eval()

    #Define softmax function
    softmax = nn.Softmax(dim = -1)
    #linear = nn.Linear(1000, 11).to(device)

    flag = 0
    pseudo_set = None
    #Iterate over the dataset by batches
    for batch in tqdm(data_loader):
        img, _ = batch

        #Forward the data
        with torch.no_grad():
            logits = model(img.to(device))
        
        probs = softmax(logits)
        #print(probs.shape)

        for i in range(len(probs)):
            #if model give the picture high confidence
            m = torch.max(probs[i])
            if m > threshold and flag == 1:
                label = (probs[i] == m).nonzero().item()
                new_labeled_data = pDataset(img[i], label)
                pseudo_set = ConcatDataset([pseudo_set, new_labeled_data])
            elif m > threshold and flag == 0:
                label = (probs[i] == m).nonzero().item()
                pseudo_set = pDataset(img[i], label)
                flag = 1

    model.train()
    return pseudo_set

In [6]:
import sys
sys.setrecursionlimit(100000)

In [7]:
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torchvision.models.resnet18(weights='ResNet18_Weights.DEFAULT')
#linear = nn.Linear(1000, 11).to(device)

model.to(device)

#For the classification task, we use cross-entropy as the measurement of performance
criterion = nn.CrossEntropyLoss()

#Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.0003, weight_decay = 1e-5, momentum = 0.9)
optimizer = torch.optim.RAdam(model.parameters(), lr = 0.0003)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 20, gamma=0.1, last_epoch=-1)

#the number of training epochs
#n_epochs = 120
n_epochs = 40

#Whether to do semi-supervised learning
do_semi = True

train_proc = []
eval_proc = []
best_acc = 0

#pseudo_set = None


for epoch in range(n_epochs):
    pseudo_set = None
    if do_semi:
        pseudo_set = get_pseudo_labels(unlabeled_set, model)
        #print(len(pseudo_set))

    if not pseudo_set == None:
        #print(len(train_set))
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        #print(len(concat_dataset))
        train_loader = DataLoader(concat_dataset, batch_size = batch_size, shuffle = True, drop_last = True, num_workers = 8)
    else:
        train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True, drop_last = True, num_workers = 8)
        
    model.train()


    train_loss = []
    train_accs = []
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(imgs)
        
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm = 10)
        optimizer.step()
        acc = (logits.argmax(dim = -1) == labels).float().mean()
        train_loss.append(loss.item())
        train_accs.append(acc)
    # The average loss and accuracy of the training set is the average of the recorded values.
    scheduler.step()
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    train_proc.append(train_loss)

    # Print the information.
    print(f"[ Train | {epoch :03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):
        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs)
        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels)

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim= -1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
    
    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    eval_proc.append(valid_loss)

    # Print the information.
    print(f"[ Valid | {epoch :03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    
    if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save(model.state_dict(), f'./pretrain/food11_{epoch}.pt')

100%|██████████| 28/28 [00:05<00:00,  4.82it/s]


[ Train | 000/040 ] loss = 7.73025, acc = 0.05831


100%|██████████| 5/5 [00:01<00:00,  3.25it/s]


[ Valid | 000/040 ] loss = 6.60579, acc = 0.05781


100%|██████████| 25/25 [00:05<00:00,  4.73it/s]


...

[ Valid | 038/040 ] loss = 0.81898, acc = 0.83594


100%|██████████| 76/76 [00:12<00:00,  5.99it/s]


[ Train | 039/040 ] loss = 0.01695, acc = 0.99620


100%|██████████| 5/5 [00:01<00:00,  3.27it/s]

[ Valid | 039/040 ] loss = 0.84027, acc = 0.83125


In [8]:
import matplotlib.pyplot as plt


epochs = np.arange(n_epochs)
plt.figure(0)
plt.plot(epochs, np.array(train_proc), color = 'b', label = 'training loss')
plt.plot(epochs, np.array(eval_proc), color = 'r', label = 'Evaluation loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig(f'loss_surface.png')
plt.clf()
plt.show()

<Figure size 432x288 with 0 Axes>

In [11]:
model = torchvision.models.resnet18()
model.to(device)
checkpoint = torch.load('pretrain/food11_28.pt')
model.load_state_dict(checkpoint)


model.eval()

predictions = []

for batch in tqdm(test_loader):
    imgs, labels = batch
    softmax = nn.Softmax(dim = -1)

    with torch.no_grad():
        logits = model(imgs.to(device))
        #print(softmax(logits))

    predictions.extend(logits.argmax(dim = 1).cpu().numpy().tolist())

100%|██████████| 27/27 [00:16<00:00,  1.63it/s]


In [12]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")